### Cleaning Procedure for ALL_DATA DB
Taking the data from the CDC website- https://www.archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008

The dataset represents ten years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks.<br>
Each row concerns hospital records of patients diagnosed with diabetes, who underwent laboratory, medications, and stayed up to 14 days.

ALL_DATA DB is the **diabetic_data.csv** file- which will be cleaned for a further analysis and visualizations.<br>
The **IDS_mapping.csv** file is a mapping between numeric categorical columns and their meaning.

In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Connecting to the ALL_DATA DB

In [ ]:
# Initialize the BigQuery client
client = bigquery.Client()

# Define your query
query = """
SELECT *
FROM `diabetes-439215.DIABETES_ALL.ALL_DATA`
"""

# Run the query and convert the result to a pandas DataFrame
df = client.query(query).to_dataframe()

# Display the first few rows of the DataFrame
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,15245874,7436034,Caucasian,Female,[80-90),?,1,3,1,8,...,False,No,No,False,False,False,False,No,True,NO
1,44118072,20660841,Other,Female,[80-90),?,6,1,17,3,...,False,No,No,False,False,False,False,No,True,NO
2,46242192,10993320,Caucasian,Female,[70-80),?,1,1,7,5,...,False,No,No,False,False,False,False,No,False,NO
3,99745974,24640821,Caucasian,Female,[60-70),?,2,11,4,3,...,False,No,No,False,False,False,False,No,True,NO
4,117268386,23872788,Caucasian,Female,[60-70),?,3,1,1,7,...,False,No,No,False,False,False,False,No,False,>30


Calculating percentage of null ('?') cells in each column

In [ ]:
# replaces '?' value with NULL
df.replace('?', np.nan, inplace=True)

#find percentage of NULL values in each column
null_percentage = (df.isnull().mean() * 100).round(2)

# Add '%' sign to each percentage value
null_percentage = null_percentage.astype(str) + '%'

# Display the percentage of null values
print(null_percentage)

encounter_id                  0.0%
patient_nbr                   0.0%
race                         2.23%
gender                        0.0%
age                           0.0%
weight                      96.86%
admission_type_id             0.0%
discharge_disposition_id      0.0%
admission_source_id           0.0%
time_in_hospital              0.0%
payer_code                  39.57%
medical_specialty           49.08%
num_lab_procedures            0.0%
num_procedures                0.0%
num_medications               0.0%
number_outpatient             0.0%
number_emergency              0.0%
number_inpatient              0.0%
diag_1                       0.02%
diag_2                       0.35%
diag_3                        1.4%
number_diagnoses              0.0%
max_glu_serum                 0.0%
A1Cresult                     0.0%
metformin                     0.0%
repaglinide                   0.0%
nateglinide                   0.0%
chlorpropamide                0.0%
glimepiride         

**The following columns should be in the same way of writing-**<br>
'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin',  
'glipizide-metformin','glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone'<br><br>
The feature indicates whether the drug was prescribed or there was a change in the dosage. <br>
**Values-**
- **up** if the dosage was increased during the encounter
- **down** if the dosage was decreased
- **steady** if the dosage did not change
- **no** if the drug was not prescribed

There are some columns with **False** value instead of **NULL**. I'll change every False to **NULL**

In [ ]:
df.miglitol.value_counts()

,count
miglitol,
False,101710


In [ ]:
df.acarbose.value_counts()

,count
acarbose,
No,101403
Steady,294
Up,10
Down,3


In [ ]:
columns_to_modify = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin',
'glipizide-metformin','glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']

# Replace 'False' with NaN in the selected columns
df[columns_to_modify] = df[columns_to_modify].replace(False, np.nan)

# Display the modified DataFrame
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,15245874,7436034,Caucasian,Female,[80-90),NaN,1,3,1,8,...,<NA>,No,No,<NA>,<NA>,<NA>,<NA>,No,True,NO
1,44118072,20660841,Other,Female,[80-90),NaN,6,1,17,3,...,<NA>,No,No,<NA>,<NA>,<NA>,<NA>,No,True,NO
2,46242192,10993320,Caucasian,Female,[70-80),NaN,1,1,7,5,...,<NA>,No,No,<NA>,<NA>,<NA>,<NA>,No,False,NO
3,99745974,24640821,Caucasian,Female,[60-70),NaN,2,11,4,3,...,<NA>,No,No,<NA>,<NA>,<NA>,<NA>,No,True,NO
4,117268386,23872788,Caucasian,Female,[60-70),NaN,3,1,1,7,...,<NA>,No,No,<NA>,<NA>,<NA>,<NA>,No,False,>30


In [ ]:
null_percentage = (df.isnull().mean() * 100).round(2)

# Add '%' sign to each percentage value
null_percentage = null_percentage.astype(str) + '%'

# Display the percentage of null values
print(null_percentage)

encounter_id                  0.0%
patient_nbr                   0.0%
race                         2.23%
gender                        0.0%
age                           0.0%
weight                      96.86%
admission_type_id             0.0%
discharge_disposition_id      0.0%
admission_source_id           0.0%
time_in_hospital              0.0%
payer_code                  39.57%
medical_specialty           49.08%
num_lab_procedures            0.0%
num_procedures                0.0%
num_medications               0.0%
number_outpatient             0.0%
number_emergency              0.0%
number_inpatient              0.0%
diag_1                       0.02%
diag_2                       0.35%
diag_3                        1.4%
number_diagnoses              0.0%
max_glu_serum                 0.0%
A1Cresult                     0.0%
metformin                     0.0%
repaglinide                   0.0%
nateglinide                   0.0%
chlorpropamide                0.0%
glimepiride         

Columns with all **NULL** values don't give me any insight with my analysis so I'll drop them.

In [ ]:
# drop all colums with 100% of null values
df.dropna(axis=1, how='all', inplace=True)
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,pioglitazone,rosiglitazone,acarbose,troglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,15245874,7436034,Caucasian,Female,[80-90),NaN,1,3,1,8,...,No,No,No,No,No,No,No,No,True,NO
1,44118072,20660841,Other,Female,[80-90),NaN,6,1,17,3,...,No,No,No,No,No,No,No,No,True,NO
2,46242192,10993320,Caucasian,Female,[70-80),NaN,1,1,7,5,...,No,No,No,No,No,No,No,No,False,NO
3,99745974,24640821,Caucasian,Female,[60-70),NaN,2,11,4,3,...,No,No,No,No,No,No,No,No,True,NO
4,117268386,23872788,Caucasian,Female,[60-70),NaN,3,1,1,7,...,No,No,No,No,No,No,No,No,False,>30


The database contains incomplete, redundant, and noisy information as expected in any real-world data. <br>
There were several features that could not be treated directly since they had a high percentage of missing values. These features were weight (96.8% values missing), payer code (39.5%), and medical specialty (49%). <br>
**Weight** attribute was considered to be too sparse and it was not included in further analysis. <br>
Large percentage of missing values of the **weight** attribute can be explained by the fact that prior to the HITECH legislation of the American Reinvestment and Recovery Act in 2009 hospitals and clinics were <u>not required to capture</u> it in a structured format.<br>
**Payer code** was removed since it had a high percentage of missing values and it was not considered relevant to the outcome. <br>
**Medical specialty** attribute was maintained, adding the value “missing” in order to account for missing values. <br>


In [ ]:
# List of columns you want to drop
columns_to_drop = ['weight','payer_code','medical_specialty']

# Drop the specified columns
df.drop(columns=columns_to_drop, inplace=True)

# Display the modified DataFrame
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,pioglitazone,rosiglitazone,acarbose,troglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,15245874,7436034,Caucasian,Female,[80-90),1,3,1,8,47,...,No,No,No,No,No,No,No,No,True,NO
1,44118072,20660841,Other,Female,[80-90),6,1,17,3,71,...,No,No,No,No,No,No,No,No,True,NO
2,46242192,10993320,Caucasian,Female,[70-80),1,1,7,5,54,...,No,No,No,No,No,No,No,No,False,NO
3,99745974,24640821,Caucasian,Female,[60-70),2,11,4,3,45,...,No,No,No,No,No,No,No,No,True,NO
4,117268386,23872788,Caucasian,Female,[60-70),3,1,1,7,45,...,No,No,No,No,No,No,No,No,False,>30


Removing duplicated rows

In [ ]:
df.drop_duplicates()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,pioglitazone,rosiglitazone,acarbose,troglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,15245874,7436034,Caucasian,Female,[80-90),1,3,1,8,47,...,No,No,No,No,No,No,No,No,True,NO
1,44118072,20660841,Other,Female,[80-90),6,1,17,3,71,...,No,No,No,No,No,No,No,No,True,NO
2,46242192,10993320,Caucasian,Female,[70-80),1,1,7,5,54,...,No,No,No,No,No,No,No,No,False,NO
3,99745974,24640821,Caucasian,Female,[60-70),2,11,4,3,45,...,No,No,No,No,No,No,No,No,True,NO
4,117268386,23872788,Caucasian,Female,[60-70),3,1,1,7,45,...,No,No,No,No,No,No,No,No,False,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101705,47596488,6758748,Hispanic,Male,[10-20),3,1,1,2,11,...,No,No,No,No,No,Steady,No,No,True,NO
101706,53195376,19884852,Caucasian,Male,[20-30),1,18,7,2,47,...,No,No,No,No,No,Steady,No,No,True,NO
101707,87102768,3539961,AfricanAmerican,Male,[40-50),1,6,7,2,50,...,No,No,No,No,No,Steady,No,No,True,>30
101708,112420782,62386227,Caucasian,Male,[0-10),2,1,1,1,9,...,No,No,No,No,No,Steady,No,No,True,NO


I'll remove all encounters that resulted in either **discharge to a hospice or patient death**, to avoid biasing the analysis.

This data will be on the **discharge_disposition_id** column- that by the data provided with the CDC (IDS_mapping.csv file), will be with IDs- 11,13,14,19,20,21.<br>
Also, unknown or NULL mapping won't be good for the analysis, so I'll remove it too- IDs 18,25,26.

In [ ]:
# Define the values in the discharge_disposition_id column that you want to drop
values_to_drop = [11,13,14,19,20,21,18,25,26]  # Replace with the actual values

# Drop rows where discharge_disposition_id matches any of the specified values
df = df[~df['discharge_disposition_id'].isin(values_to_drop)]

# Display the modified DataFrame
df

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,pioglitazone,rosiglitazone,acarbose,troglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,15245874,7436034,Caucasian,Female,[80-90),1,3,1,8,47,...,No,No,No,No,No,No,No,No,True,NO
1,44118072,20660841,Other,Female,[80-90),6,1,17,3,71,...,No,No,No,No,No,No,No,No,True,NO
2,46242192,10993320,Caucasian,Female,[70-80),1,1,7,5,54,...,No,No,No,No,No,No,No,No,False,NO
4,117268386,23872788,Caucasian,Female,[60-70),3,1,1,7,45,...,No,No,No,No,No,No,No,No,False,>30
5,133884690,93890304,Caucasian,Male,[50-60),1,1,7,4,86,...,No,No,No,No,No,No,No,No,False,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101704,38352174,17944479,Caucasian,Male,[10-20),1,1,7,2,64,...,No,No,No,No,No,Steady,No,No,True,NO
101705,47596488,6758748,Hispanic,Male,[10-20),3,1,1,2,11,...,No,No,No,No,No,Steady,No,No,True,NO
101707,87102768,3539961,AfricanAmerican,Male,[40-50),1,6,7,2,50,...,No,No,No,No,No,Steady,No,No,True,>30
101708,112420782,62386227,Caucasian,Male,[0-10),2,1,1,1,9,...,No,No,No,No,No,Steady,No,No,True,NO


I want to have independent observations- so if a patient was readmitted after less than 30 days, the observation would not be independent.<br>
I'll remove observations with such readmission and keep only the first encounter.<br>
For patients that were readmitted after more than 30 days, I'll create a new ID, as they should be considered as independent patients.

In [ ]:
# Sort the DataFrame by 'encounter_id'
df.sort_values(by='encounter_id', inplace=True)

# Reset the index to start from 1
df.reset_index(drop=True, inplace=True)

# Create a new 'encounter_id' column starting from 1
df['encounter_id'] = df.index + 1

# Display the modified DataFrame
df

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,pioglitazone,rosiglitazone,acarbose,troglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,1,48330783,Caucasian,Female,[80-90),2,1,4,13,68,...,No,No,No,No,No,Steady,No,Ch,True,NO
1,2,63555939,Caucasian,Female,[90-100),3,3,4,12,33,...,No,Steady,No,No,No,Steady,No,Ch,True,NO
2,3,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,No,No,No,No,Steady,No,Ch,True,NO
3,4,89869032,AfricanAmerican,Female,[40-50),1,1,7,9,47,...,No,No,No,No,No,Steady,No,No,True,>30
4,5,82637451,Caucasian,Male,[50-60),2,1,2,3,31,...,No,No,No,No,No,Steady,No,No,True,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94604,94605,100162476,AfricanAmerican,Male,[70-80),1,3,7,3,51,...,No,No,No,No,No,Down,No,Ch,True,>30
94605,94606,74694222,AfricanAmerican,Female,[80-90),1,4,5,5,33,...,No,No,No,No,No,Steady,No,No,True,NO
94606,94607,41088789,Caucasian,Male,[70-80),1,1,7,1,53,...,No,No,No,No,No,Down,No,Ch,True,NO
94607,94608,31693671,Caucasian,Female,[80-90),2,3,7,10,45,...,Steady,No,No,No,No,Up,No,Ch,True,NO


In [ ]:
# Find duplicate patient_id values
duplicate_patient_ids = df[df.duplicated(subset='patient_nbr', keep=False)]

# Display the rows with duplicate patient_id values
print(duplicate_patient_ids[['patient_nbr']].drop_duplicates())

       patient_nbr
6         85504905
25       115196778
26        41606064
30        80845353
31       114715242
...            ...
93735    141300419
93767     32337963
93844     89955270
94189     44193978
94209    180739895

[15352 rows x 1 columns]


In [ ]:
df[df['patient_nbr']==85504905]

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,pioglitazone,rosiglitazone,acarbose,troglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
6,7,85504905,Caucasian,Female,[40-50),1,3,7,7,60,...,No,No,No,No,No,Down,No,Ch,True,<30
17342,17343,85504905,Caucasian,Female,[40-50),1,6,7,9,46,...,No,No,No,No,No,Steady,No,No,True,>30


In [ ]:
# Identify rows with duplicate patient_nbr and readmitted='<30' on smaller encounter_id
duplicate_patients = df[df.duplicated(subset='patient_nbr', keep=False) & df['readmitted'].isin(['<30'])]

# Get the encounter_id of the rows to remove (those with readmitted='<30')
encounter_ids_to_remove = duplicate_patients.groupby('patient_nbr')['encounter_id'].min().values

# Filter the DataFrame to keep only the desired rows
df = df[~((df.duplicated(subset='patient_nbr', keep=False)) & (~df['encounter_id'].isin(encounter_ids_to_remove)))]
df

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,pioglitazone,rosiglitazone,acarbose,troglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,1,48330783,Caucasian,Female,[80-90),2,1,4,13,68,...,No,No,No,No,No,Steady,No,Ch,True,NO
1,2,63555939,Caucasian,Female,[90-100),3,3,4,12,33,...,No,Steady,No,No,No,Steady,No,Ch,True,NO
2,3,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,No,No,No,No,Steady,No,Ch,True,NO
3,4,89869032,AfricanAmerican,Female,[40-50),1,1,7,9,47,...,No,No,No,No,No,Steady,No,No,True,>30
4,5,82637451,Caucasian,Male,[50-60),2,1,2,3,31,...,No,No,No,No,No,Steady,No,No,True,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94597,94598,183087545,Caucasian,Female,[70-80),1,1,7,9,50,...,No,No,No,No,No,Steady,No,Ch,True,>30
94598,94599,188574944,Other,Female,[40-50),1,1,7,14,73,...,No,No,No,No,No,Up,No,Ch,True,>30
94599,94600,140199494,Other,Female,[60-70),1,1,7,2,46,...,No,No,No,No,No,Steady,No,No,True,>30
94601,94602,120975314,Caucasian,Female,[80-90),1,1,7,5,76,...,No,No,No,No,No,Up,No,Ch,True,NO


Change values for the '**change**' column- Indicates if there was a change in diabetic medications (either dosage or medication) <br>
- **No** value to False
- **Ch** value to True

In [ ]:
df['change'] = df['change'].replace({'No': False, 'Ch': True})
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,pioglitazone,rosiglitazone,acarbose,troglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,1,48330783,Caucasian,Female,[80-90),2,1,4,13,68,...,No,No,No,No,No,Steady,No,True,True,NO
1,2,63555939,Caucasian,Female,[90-100),3,3,4,12,33,...,No,Steady,No,No,No,Steady,No,True,True,NO
2,3,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,No,No,No,No,Steady,No,True,True,NO
3,4,89869032,AfricanAmerican,Female,[40-50),1,1,7,9,47,...,No,No,No,No,No,Steady,No,False,True,>30
4,5,82637451,Caucasian,Male,[50-60),2,1,2,3,31,...,No,No,No,No,No,Steady,No,False,True,>30


In [ ]:
# prompt: drop rows with null values

df.dropna(inplace=True)

In [ ]:
df

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,pioglitazone,rosiglitazone,acarbose,troglitazone,tolazamide,insulin,glyburide-metformin,change,diabetesMed,readmitted
0,1,48330783,Caucasian,Female,[80-90),2,1,4,13,68,...,No,No,No,No,No,Steady,No,True,True,NO
1,2,63555939,Caucasian,Female,[90-100),3,3,4,12,33,...,No,Steady,No,No,No,Steady,No,True,True,NO
2,3,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,No,No,No,No,Steady,No,True,True,NO
3,4,89869032,AfricanAmerican,Female,[40-50),1,1,7,9,47,...,No,No,No,No,No,Steady,No,False,True,>30
4,5,82637451,Caucasian,Male,[50-60),2,1,2,3,31,...,No,No,No,No,No,Steady,No,False,True,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94597,94598,183087545,Caucasian,Female,[70-80),1,1,7,9,50,...,No,No,No,No,No,Steady,No,True,True,>30
94598,94599,188574944,Other,Female,[40-50),1,1,7,14,73,...,No,No,No,No,No,Up,No,True,True,>30
94599,94600,140199494,Other,Female,[60-70),1,1,7,2,46,...,No,No,No,No,No,Steady,No,False,True,>30
94601,94602,120975314,Caucasian,Female,[80-90),1,1,7,5,76,...,No,No,No,No,No,Up,No,True,True,NO


Saving the cleaned DataFrame as a DB in BigQuery

In [ ]:
from google.cloud import bigquery

# Function to map Pandas dtypes to BigQuery schema fields
def generate_bq_schema(df):
    schema = []

    # Map Pandas dtypes to BigQuery types
    for col_name, dtype in zip(df.columns, df.dtypes):
        if pd.api.types.is_string_dtype(dtype):
            schema.append(bigquery.SchemaField(col_name, "STRING"))
        elif pd.api.types.is_numeric_dtype(dtype):
            if pd.api.types.is_integer_dtype(dtype):
                schema.append(bigquery.SchemaField(col_name, "INTEGER"))
            else:
                schema.append(bigquery.SchemaField(col_name, "FLOAT"))
        elif pd.api.types.is_bool_dtype(dtype):
            schema.append(bigquery.SchemaField(col_name, "BOOLEAN"))
        elif pd.api.types.is_datetime64_any_dtype(dtype):
            schema.append(bigquery.SchemaField(col_name, "TIMESTAMP"))
        else:
            # Default to STRING if dtype is not specifically handled
            schema.append(bigquery.SchemaField(col_name, "STRING"))

    return schema

# Generate schema from the DataFrame
schema = generate_bq_schema(df)

# Print the schema
for field in schema:
    print(f"{field.name}: {field.field_type}")

encounter_id: INTEGER
patient_nbr: INTEGER
race: STRING
gender: STRING
age: STRING
admission_type_id: INTEGER
discharge_disposition_id: INTEGER
admission_source_id: INTEGER
time_in_hospital: INTEGER
num_lab_procedures: INTEGER
num_procedures: INTEGER
num_medications: INTEGER
number_outpatient: INTEGER
number_emergency: INTEGER
number_inpatient: INTEGER
diag_1: STRING
diag_2: STRING
diag_3: STRING
number_diagnoses: INTEGER
max_glu_serum: STRING
A1Cresult: STRING
metformin: STRING
repaglinide: STRING
nateglinide: STRING
chlorpropamide: STRING
glimepiride: STRING
glipizide: STRING
glyburide: STRING
tolbutamide: STRING
pioglitazone: STRING
rosiglitazone: STRING
acarbose: STRING
troglitazone: STRING
tolazamide: STRING
insulin: STRING
glyburide-metformin: STRING
change: FLOAT
diabetesMed: FLOAT
readmitted: STRING


In [ ]:
# Define the destination table ID (project_id.dataset_id.table_id)
project_id = 'diabetes-439215'
dataset_id = 'DIABETES_ALL'
table_id = 'DATA_CLEAN'

# Full table reference
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Create a new BigQuery table with the generated schema
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # Create the table
print(f"Created table {table_ref}")

# Insert the DataFrame into the new table
job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_APPEND)

job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Wait for the job to complete

print(f"Loaded {df.shape[0]} rows into {table_ref}.")

Created table diabetes-439215.DIABETES_ALL.DATA_CLEAN
Loaded 57003 rows into diabetes-439215.DIABETES_ALL.DATA_CLEAN.


Saving the Cleaned DF as a CSV file to import it to Cognos for visualizations

In [ ]:
# prompt: export df to a csv file with the headers

df.to_csv('dibetes_clean_data.csv', index=False)